# Battle of Neighborhoods Part II

### In this notebook, we are going to check whether opening an 'Indian Restaurant' in the city of Edmonton, Alberts, Canada will be a profitable idea or not

#### Importing all the required libraries

In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
# import k-means from clustering stage
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
print('All libraries imported.')

All libraries imported.


#### Loading the data of Postal Codes of Alberta, Canada from the wikipedia page

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T')[1]

In [3]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


#### Checking the data types

In [4]:
df.dtypes

Postal Code     object
Borough         object
Neighborhood    object
Latitude        object
Longitude       object
dtype: object

#### Cleaning the data by removing rows where Borough, Latitude or Longitude are not assigned

In [5]:
df.drop(df[df["Borough"]=="Not assigned"].index, axis=0, inplace=True)

In [6]:
df.drop(df[df["Latitude"]=="Not assigned"].index, axis=0, inplace=True)

In [7]:
df.drop(df[df["Longitude"]=="Not assigned"].index, axis=0, inplace=True)

#### Joining the rows with same postal code

In [8]:
df_new=df.groupby('Postal Code')['Neighborhood'].apply(','.join).reset_index()

In [9]:
PostalCode_Canada = pd.merge(df, df_new, on='Postal Code', how='inner')

In [10]:
PostalCode_Canada.drop(['Neighborhood_y'], axis=1, inplace=True)

In [11]:
PostalCode_Canada.rename(columns={'Neighborhood_x':'Neighborhood'}, inplace=True)

In [12]:
PostalCode_Canada['Neighborhood'] = np.where(PostalCode_Canada['Neighborhood'] == 'Not assigned', PostalCode_Canada['Borough'], PostalCode_Canada['Neighborhood'])
PostalCode_Canada.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


#### Converting Latitude and Longitude columns to numeric data type for further investigations

In [13]:
PostalCode_Canada.Latitude = pd.to_numeric(PostalCode_Canada.Latitude)

In [14]:
PostalCode_Canada.Longitude = pd.to_numeric(PostalCode_Canada.Longitude)

#### Shape of the final dataset

In [15]:
PostalCode_Canada.shape

(138, 5)

#### Let's check all the neighborhoods of Alberta on a map using folium

In [16]:
# Let's first get the coordinates of Alberta using Nominatim and name our user_agent as 'ca_explorer'
address = 'Alberta'

geolocator = Nominatim(user_agent="Ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Alberta are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Alberta are 55.001251, -115.002136.


In [17]:
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(PostalCode_Canada['Latitude'], PostalCode_Canada['Longitude'], PostalCode_Canada['Borough'], PostalCode_Canada['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

#### Since we aremore interested in opening the restaurant in Edmonton, let's fetch the data with only Edmonton as borough from our dataframe

In [18]:
Edmonton_data = PostalCode_Canada[PostalCode_Canada['Borough'].str.contains('Edmonton')].reset_index(drop=True)
Edmonton_data.shape
Edmonton_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
1,T6A,Edmonton,North Capilano,53.5483,-113.4080
2,T5B,Edmonton,"East North Central, West Beverly",53.5766,-113.4608
3,T6B,Edmonton,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404
4,T5C,Edmonton,Central Londonderry,53.6129,-113.4572
5,T6C,Edmonton,Central Bonnie Doon,53.5182,-113.4769
6,T5E,Edmonton,"West Londonderry, East Calder",53.5923,-113.5168
7,T6E,Edmonton,"South Bonnie Doon, East University",53.5087,-113.5078
8,T5G,Edmonton,"North Central, Queen Mary Park, Blatchford",53.5682,-113.4822
9,T6G,Edmonton,"West University, Strathcona Place",53.5248,-113.5334


#### As we can see from the shape of Edmonton_data, there are 38 neighborhoods with Edmonton as the borough. 
#### Let's check all those neighborhoods of Edmonton superimposed on top of the map using folium

In [19]:
# create map of Toronto using latitude and longitude values
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Edmonton_data['Latitude'], Edmonton_data['Longitude'], Edmonton_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edmonton)  
    
map_edmonton

#### Now we need to chose the neighborhood which has the maximum Indian population

#### From the following link, it is seen that 'Mill Woods' is the neighborhood where most of the South Asian community resides. https://citymuseumedmonton.ca/2016/11/22/edmontons-south-asian-community/

#### so let's explore that neighbourhood further using Foursquare App

In [20]:
CLIENT_ID = 'I441WKWA3IK5UAYFHFDTQMBXLAQQVLBRXI1LSCKUEJ32ZYHV' # your Foursquare ID
CLIENT_SECRET = 'OCU5ASUTUHG03SOWPJW1CZCWDEHAC4T5L2YB1AOPNHIOAEOZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I441WKWA3IK5UAYFHFDTQMBXLAQQVLBRXI1LSCKUEJ32ZYHV
CLIENT_SECRET:OCU5ASUTUHG03SOWPJW1CZCWDEHAC4T5L2YB1AOPNHIOAEOZ


In [21]:
# fetching Mill woods data from our Edmonton_data
Edmonton_millwoods = Edmonton_data[Edmonton_data['Neighborhood'].str.contains('Mill Woods')].reset_index(drop=True)
Edmonton_millwoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T6K,Edmonton,West Mill Woods,53.4816,-113.4623
1,T6L,Edmonton,East Mill Woods,53.4681,-113.4339


#### Let's check 'Indian Restaurants' in the East Mill Woods area using Foursquare

In [22]:
address = 'East mill Woods, Edmonton, Alberta'

geolocator = Nominatim(user_agent="Ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Mill Woods are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Mill Woods are 53.460272, -113.428141.


In [23]:
search_query = 'Indian Restaurant'
radius = 2500

In [24]:
# define the url
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=I441WKWA3IK5UAYFHFDTQMBXLAQQVLBRXI1LSCKUEJ32ZYHV&client_secret=OCU5ASUTUHG03SOWPJW1CZCWDEHAC4T5L2YB1AOPNHIOAEOZ&ll=53.460272,-113.428141&v=20180605&query=Indian Restaurant&radius=2500&limit=100'

In [25]:
# getting the json file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd56e7fb6bd2d4f4bda5203'},
 'response': {'venues': [{'id': '4b075e2cf964a520bcfc22e3',
    'name': 'Aashiana Indian Restaurant',
    'location': {'address': '1527 50 St NW',
     'crossStreet': 'Millwoods Road South',
     'lat': 53.445077,
     'lng': -113.418912,
     'labeledLatLngs': [{'label': 'display',
       'lat': 53.445077,
       'lng': -113.418912}],
     'distance': 1798,
     'postalCode': 'T6L 5N9',
     'cc': 'CA',
     'city': 'Edmonton',
     'state': 'AB',
     'country': 'Canada',
     'formattedAddress': ['1527 50 St NW (Millwoods Road South)',
      'Edmonton AB T6L 5N9',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607822975',
    'hasPerk': Fal

In [26]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-26-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4b075e2cf964a520bcfc22e3,Aashiana Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822975,False,1527 50 St NW,Millwoods Road South,53.445077,-113.418912,"[{'label': 'display', 'lat': 53.445077, 'lng':...",1798,T6L 5N9,CA,Edmonton,AB,Canada,"[1527 50 St NW (Millwoods Road South), Edmonto..."
1,4fe7f9c5e4b01b7b6c9d39a8,Tandoori King Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822975,False,NaN,NaN,53.445071,-113.418404,"[{'label': 'display', 'lat': 53.44507061107696...",1811,NaN,CA,NaN,NaN,Canada,[Canada]
2,5b614c641ffe97002c654b4c,Indian Astrologer & Psychic - Pandith Rajaram,"[{'id': '52f2ab2ebcbc57f1066b8b43', 'name': 'A...",v-1607822975,False,4540 25A Ave NW,NaN,53.457232,-113.412227,"[{'label': 'display', 'lat': 53.457232, 'lng':...",1107,T6L 3X4,CA,Edmonton,AB,Canada,"[4540 25A Ave NW, Edmonton AB T6L 3X4, Canada]"
3,5220e79211d27824c02a8f8f,New Indian Sweets & Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822975,False,NaN,NaN,53.458772,-113.392743,"[{'label': 'display', 'lat': 53.45877159724264...",2352,NaN,CA,NaN,NaN,Canada,[Canada]
4,4b79b09af964a5205b0b2fe3,Rosie's family restaurant,[],v-1607822975,False,6108 28ave,NaN,53.455700,-113.424996,"[{'label': 'display', 'lat': 53.45569956930718...",550,NaN,CA,NaN,Alberta,Canada,"[6108 28ave, Alberta, Canada]"


In [27]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Aashiana Indian Restaurant,Indian Restaurant,1527 50 St NW,Millwoods Road South,53.445077,-113.418912,"[{'label': 'display', 'lat': 53.445077, 'lng':...",1798,T6L 5N9,CA,Edmonton,AB,Canada,"[1527 50 St NW (Millwoods Road South), Edmonto...",4b075e2cf964a520bcfc22e3
1,Tandoori King Restaurant,Indian Restaurant,NaN,NaN,53.445071,-113.418404,"[{'label': 'display', 'lat': 53.44507061107696...",1811,NaN,CA,NaN,NaN,Canada,[Canada],4fe7f9c5e4b01b7b6c9d39a8
2,Indian Astrologer & Psychic - Pandith Rajaram,Astrologer,4540 25A Ave NW,NaN,53.457232,-113.412227,"[{'label': 'display', 'lat': 53.457232, 'lng':...",1107,T6L 3X4,CA,Edmonton,AB,Canada,"[4540 25A Ave NW, Edmonton AB T6L 3X4, Canada]",5b614c641ffe97002c654b4c
3,New Indian Sweets & Restaurant,Indian Restaurant,NaN,NaN,53.458772,-113.392743,"[{'label': 'display', 'lat': 53.45877159724264...",2352,NaN,CA,NaN,NaN,Canada,[Canada],5220e79211d27824c02a8f8f
4,Rosie's family restaurant,None,6108 28ave,NaN,53.455700,-113.424996,"[{'label': 'display', 'lat': 53.45569956930718...",550,NaN,CA,NaN,Alberta,Canada,"[6108 28ave, Alberta, Canada]",4b79b09af964a5205b0b2fe3
5,Dairy Queen,Ice Cream Shop,"5403 23rd Ave,Millwoods Town Centre",NaN,53.453364,-113.425041,"[{'label': 'display', 'lat': 53.45336409999999...",795,T6L 7G4,CA,Edmonton,AB,Canada,"[5403 23rd Ave,Millwoods Town Centre, Edmonton...",591491a28173cb55bcb79058
6,Pandith Ramji - Indian Astrologer,Astrologer,7413 32nd Ave NW,NaN,53.463330,-113.447517,"[{'label': 'display', 'lat': 53.46333, 'lng': ...",1328,T6A 1J9,CA,Edmonton,AB,Canada,"[7413 32nd Ave NW, Edmonton AB T6A 1J9, Canada]",57a2434038fac2fe3109af54
7,Rasoi - Indian Kitchen,Indian Restaurant,4341,50st,53.481173,-113.416976,"[{'label': 'display', 'lat': 53.481173, 'lng':...",2441,T6L 7E8,CA,Edmonton,AB,Canada,"[4341 (50st), Edmonton AB T6L 7E8, Canada]",4c11b1b217002d7fd7c3e609
8,Turo Turo,Filipino Restaurant,NaN,NaN,53.480695,-113.443791,"[{'label': 'display', 'lat': 53.48069505222659...",2498,NaN,CA,NaN,NaN,Canada,[Canada],4f6e12c3e4b0e0075b2e9331
9,Smitty's Family Restaurant,Restaurant,5556 50 Ave,NaN,53.474089,-113.434213,"[{'label': 'display', 'lat': 53.47408878819808...",1589,T0A 3A1,CA,St Paul,AB,Canada,"[5556 50 Ave, St Paul AB T0A 3A1, Canada]",5ebc37f302a08d00088862f5


#### Performing the same steps for West Mill Woods

In [28]:
address = 'West mill Woods, Edmonton, Alberta'

geolocator = Nominatim(user_agent="Ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Mill Woods are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of West Mill Woods are 53.460272, -113.428141.


In [29]:
search_query = 'Indian Restaurant'
radius = 2500

In [30]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=I441WKWA3IK5UAYFHFDTQMBXLAQQVLBRXI1LSCKUEJ32ZYHV&client_secret=OCU5ASUTUHG03SOWPJW1CZCWDEHAC4T5L2YB1AOPNHIOAEOZ&ll=53.460272,-113.428141&v=20180605&query=Indian Restaurant&radius=2500&limit=100'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd56e8abec12724ec6bbe05'},
 'response': {'venues': [{'id': '4b075e2cf964a520bcfc22e3',
    'name': 'Aashiana Indian Restaurant',
    'location': {'address': '1527 50 St NW',
     'crossStreet': 'Millwoods Road South',
     'lat': 53.445077,
     'lng': -113.418912,
     'labeledLatLngs': [{'label': 'display',
       'lat': 53.445077,
       'lng': -113.418912}],
     'distance': 1798,
     'postalCode': 'T6L 5N9',
     'cc': 'CA',
     'city': 'Edmonton',
     'state': 'AB',
     'country': 'Canada',
     'formattedAddress': ['1527 50 St NW (Millwoods Road South)',
      'Edmonton AB T6L 5N9',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607822986',
    'hasPerk': Fal

In [32]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

<ipython-input-32-67768799b19f>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4b075e2cf964a520bcfc22e3,Aashiana Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822986,False,1527 50 St NW,Millwoods Road South,53.445077,-113.418912,"[{'label': 'display', 'lat': 53.445077, 'lng':...",1798,T6L 5N9,CA,Edmonton,AB,Canada,"[1527 50 St NW (Millwoods Road South), Edmonto..."
1,4fe7f9c5e4b01b7b6c9d39a8,Tandoori King Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822986,False,NaN,NaN,53.445071,-113.418404,"[{'label': 'display', 'lat': 53.44507061107696...",1811,NaN,CA,NaN,NaN,Canada,[Canada]
2,5b614c641ffe97002c654b4c,Indian Astrologer & Psychic - Pandith Rajaram,"[{'id': '52f2ab2ebcbc57f1066b8b43', 'name': 'A...",v-1607822986,False,4540 25A Ave NW,NaN,53.457232,-113.412227,"[{'label': 'display', 'lat': 53.457232, 'lng':...",1107,T6L 3X4,CA,Edmonton,AB,Canada,"[4540 25A Ave NW, Edmonton AB T6L 3X4, Canada]"
3,5220e79211d27824c02a8f8f,New Indian Sweets & Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822986,False,NaN,NaN,53.458772,-113.392743,"[{'label': 'display', 'lat': 53.45877159724264...",2352,NaN,CA,NaN,NaN,Canada,[Canada]
4,4b79b09af964a5205b0b2fe3,Rosie's family restaurant,[],v-1607822986,False,6108 28ave,NaN,53.455700,-113.424996,"[{'label': 'display', 'lat': 53.45569956930718...",550,NaN,CA,NaN,Alberta,Canada,"[6108 28ave, Alberta, Canada]"
5,591491a28173cb55bcb79058,Dairy Queen,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",v-1607822986,False,"5403 23rd Ave,Millwoods Town Centre",NaN,53.453364,-113.425041,"[{'label': 'display', 'lat': 53.45336409999999...",795,T6L 7G4,CA,Edmonton,AB,Canada,"[5403 23rd Ave,Millwoods Town Centre, Edmonton..."
6,57a2434038fac2fe3109af54,Pandith Ramji - Indian Astrologer,"[{'id': '52f2ab2ebcbc57f1066b8b43', 'name': 'A...",v-1607822986,False,7413 32nd Ave NW,NaN,53.463330,-113.447517,"[{'label': 'display', 'lat': 53.46333, 'lng': ...",1328,T6A 1J9,CA,Edmonton,AB,Canada,"[7413 32nd Ave NW, Edmonton AB T6A 1J9, Canada]"
7,4c11b1b217002d7fd7c3e609,Rasoi - Indian Kitchen,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1607822986,False,4341,50st,53.481173,-113.416976,"[{'label': 'display', 'lat': 53.481173, 'lng':...",2441,T6L 7E8,CA,Edmonton,AB,Canada,"[4341 (50st), Edmonton AB T6L 7E8, Canada]"
8,4f6e12c3e4b0e0075b2e9331,Turo Turo,"[{'id': '4eb1bd1c3b7b55596b4a748f', 'name': 'F...",v-1607822986,False,NaN,NaN,53.480695,-113.443791,"[{'label': 'display', 'lat': 53.48069505222659...",2498,NaN,CA,NaN,NaN,Canada,[Canada]
9,5ebc37f302a08d00088862f5,Smitty's Family Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1607822986,False,5556 50 Ave,NaN,53.474089,-113.434213,"[{'label': 'display', 'lat': 53.47408878819808...",1589,T0A 3A1,CA,St Paul,AB,Canada,"[5556 50 Ave, St Paul AB T0A 3A1, Canada]"


In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Aashiana Indian Restaurant,Indian Restaurant,1527 50 St NW,Millwoods Road South,53.445077,-113.418912,"[{'label': 'display', 'lat': 53.445077, 'lng':...",1798,T6L 5N9,CA,Edmonton,AB,Canada,"[1527 50 St NW (Millwoods Road South), Edmonto...",4b075e2cf964a520bcfc22e3
1,Tandoori King Restaurant,Indian Restaurant,NaN,NaN,53.445071,-113.418404,"[{'label': 'display', 'lat': 53.44507061107696...",1811,NaN,CA,NaN,NaN,Canada,[Canada],4fe7f9c5e4b01b7b6c9d39a8
2,Indian Astrologer & Psychic - Pandith Rajaram,Astrologer,4540 25A Ave NW,NaN,53.457232,-113.412227,"[{'label': 'display', 'lat': 53.457232, 'lng':...",1107,T6L 3X4,CA,Edmonton,AB,Canada,"[4540 25A Ave NW, Edmonton AB T6L 3X4, Canada]",5b614c641ffe97002c654b4c
3,New Indian Sweets & Restaurant,Indian Restaurant,NaN,NaN,53.458772,-113.392743,"[{'label': 'display', 'lat': 53.45877159724264...",2352,NaN,CA,NaN,NaN,Canada,[Canada],5220e79211d27824c02a8f8f
4,Rosie's family restaurant,None,6108 28ave,NaN,53.455700,-113.424996,"[{'label': 'display', 'lat': 53.45569956930718...",550,NaN,CA,NaN,Alberta,Canada,"[6108 28ave, Alberta, Canada]",4b79b09af964a5205b0b2fe3
5,Dairy Queen,Ice Cream Shop,"5403 23rd Ave,Millwoods Town Centre",NaN,53.453364,-113.425041,"[{'label': 'display', 'lat': 53.45336409999999...",795,T6L 7G4,CA,Edmonton,AB,Canada,"[5403 23rd Ave,Millwoods Town Centre, Edmonton...",591491a28173cb55bcb79058
6,Pandith Ramji - Indian Astrologer,Astrologer,7413 32nd Ave NW,NaN,53.463330,-113.447517,"[{'label': 'display', 'lat': 53.46333, 'lng': ...",1328,T6A 1J9,CA,Edmonton,AB,Canada,"[7413 32nd Ave NW, Edmonton AB T6A 1J9, Canada]",57a2434038fac2fe3109af54
7,Rasoi - Indian Kitchen,Indian Restaurant,4341,50st,53.481173,-113.416976,"[{'label': 'display', 'lat': 53.481173, 'lng':...",2441,T6L 7E8,CA,Edmonton,AB,Canada,"[4341 (50st), Edmonton AB T6L 7E8, Canada]",4c11b1b217002d7fd7c3e609
8,Turo Turo,Filipino Restaurant,NaN,NaN,53.480695,-113.443791,"[{'label': 'display', 'lat': 53.48069505222659...",2498,NaN,CA,NaN,NaN,Canada,[Canada],4f6e12c3e4b0e0075b2e9331
9,Smitty's Family Restaurant,Restaurant,5556 50 Ave,NaN,53.474089,-113.434213,"[{'label': 'display', 'lat': 53.47408878819808...",1589,T0A 3A1,CA,St Paul,AB,Canada,"[5556 50 Ave, St Paul AB T0A 3A1, Canada]",5ebc37f302a08d00088862f5


#### As we can compare, nearby areas of both East and West Mill woods have common Indian Restaurants. Let's explore all the restaurants one by one and get the Ratings usinf Foursquare

In [34]:
venue_id = '4b79b09af964a5205b0b2fe3' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4b79b09af964a5205b0b2fe3?client_id=I441WKWA3IK5UAYFHFDTQMBXLAQQVLBRXI1LSCKUEJ32ZYHV&client_secret=OCU5ASUTUHG03SOWPJW1CZCWDEHAC4T5L2YB1AOPNHIOAEOZ&v=20180605'

In [35]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


{'id': '4b79b09af964a5205b0b2fe3',
 'name': "Rosie's family restaurant",
 'contact': {},
 'location': {'address': '6108 28ave',
  'lat': 53.45569956930718,
  'lng': -113.4249961081788,
  'labeledLatLngs': [{'label': 'display',
    'lat': 53.45569956930718,
    'lng': -113.4249961081788}],
  'cc': 'CA',
  'state': 'Alberta',
  'country': 'Canada',
  'formattedAddress': ['6108 28ave', 'Alberta', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/rosies-family-restaurant/4b79b09af964a5205b0b2fe3',
 'categories': [],
 'verified': False,
 'stats': {'tipCount': 0},
 'likes': {'count': 0, 'groups': []},
 'dislike': False,
 'ok': False,
 'beenHere': {'count': 0,
  'unconfirmedCount': 0,
  'marked': False,
  'lastCheckinExpiredAt': 0},
 'specials': {'count': 0, 'items': []},
 'photos': {'count': 0, 'groups': []},
 'reasons': {'count': 0, 'items': []},
 'hereNow': {'count': 0, 'summary': 'Nobody here', 'groups': []},
 'createdAt': 1266266266,
 'tips': {'count': 0, 'groups': []},
 'shortUrl':

In [36]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [37]:
venue_id = '591491a28173cb55bcb79058' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [38]:
venue_id = '4b075e2cf964a520bcfc22e3' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [39]:
venue_id = '4fe7f9c5e4b01b7b6c9d39a8' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [40]:
venue_id = '5220e79211d27824c02a8f8f' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [41]:
venue_id = '4c11b1b217002d7fd7c3e609' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [42]:
venue_id = '5ebc37f302a08d00088862f5' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [43]:
venue_id = '5529b997498e1e86629c9eca' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


### Since no restaurant  has been rated yet, this leads us to the fact that no Indian restaurant is highly recommended by the people residing there. 

#### let's explore the other trending venues of Mill Woods and diplay them on the map to ....

In [44]:
address = 'Mill Woods, Edmonton, Alberta'

geolocator = Nominatim(user_agent="Ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mill Woods are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mill Woods are 53.460272, -113.428141.


In [45]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=I441WKWA3IK5UAYFHFDTQMBXLAQQVLBRXI1LSCKUEJ32ZYHV&client_secret=OCU5ASUTUHG03SOWPJW1CZCWDEHAC4T5L2YB1AOPNHIOAEOZ&ll=53.460272,-113.428141&v=20180605&radius=2500&limit=100'

In [46]:
results = requests.get(url).json()
'There are {} in the Mill Woods.'.format(len(results['response']['groups'][0]['items']))

'There are 45 in the Mill Woods.'

In [47]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '51f7defe498e7e2d833c0aa6',
  'name': 'Memphis Blues Barbeque House',
  'location': {'address': '5317 - 23 Ave NW',
   'lat': 53.453618159736706,
   'lng': -113.42359141136835,
   'labeledLatLngs': [{'label': 'display',
     'lat': 53.453618159736706,
     'lng': -113.42359141136835}],
   'distance': 799,
   'postalCode': 'T6L 7G4',
   'cc': 'CA',
   'city': 'Edmonton',
   'state': 'AB',
   'country': 'Canada',
   'formattedAddress': ['5317 - 23 Ave NW', 'Edmonton AB T6L 7G4', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1df931735',
    'name': 'BBQ Joint',
    'pluralName': 'BBQ Joints',
    'shortName': 'BBQ',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-51f7defe498e7e2d833c0aa6-0

In [48]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered

<ipython-input-48-f882b597056b>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(items) # flatten JSON


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Memphis Blues Barbeque House,BBQ Joint,5317 - 23 Ave NW,53.453618,-113.423591,"[{'label': 'display', 'lat': 53.45361815973670...",799,T6L 7G4,CA,Edmonton,AB,Canada,"[5317 - 23 Ave NW, Edmonton AB T6L 7G4, Canada]",NaN,NaN,51f7defe498e7e2d833c0aa6
1,Booster Juice,Smoothie Shop,6136 - 28 Ave,53.459330,-113.428022,"[{'label': 'display', 'lat': 53.45933045, 'lng...",105,T6L 6N4,CA,Edmonton,AB,Canada,"[6136 - 28 Ave, Edmonton AB T6L 6N4, Canada]",NaN,NaN,4bc49629920eb71334ed1e2c
2,GoodLife Fitness Edmonton Mill Woods Town Centre,Gym / Fitness Center,2331 66 St NW,53.455768,-113.432244,"[{'label': 'display', 'lat': 53.455768, 'lng':...",570,T6K 4B4,CA,Edmonton,AB,Canada,"[2331 66 St NW, Edmonton AB T6K 4B4, Canada]",NaN,NaN,514de105e4b0fff02e76be48
3,Starbucks,Coffee Shop,5808 23 Ave,53.454295,-113.429831,"[{'label': 'display', 'lat': 53.45429488054439...",674,T6K 4B5,CA,Edmonton,AB,Canada,"[5808 23 Ave (beside Mill Woods Town Centre), ...",beside Mill Woods Town Centre,NaN,4c06c467b4aa0f47fe9e6462
4,Noodle Xtreme,Vietnamese Restaurant,2605 - 66 St,53.456888,-113.431769,"[{'label': 'display', 'lat': 53.45688844913341...",446,NaN,CA,Edmonton,AB,Canada,"[2605 - 66 St, Edmonton AB, Canada]",NaN,NaN,4c0aee84ffb8c9b6e6836c61
5,Sobeys - Millwoods Common,Grocery Store,5011 23 Ave,53.453775,-113.421703,"[{'label': 'display', 'lat': 53.453775, 'lng':...",839,T6L 7G5,CA,Edmonton,AB,Canada,"[5011 23 Ave (Mill Woods Common), Edmonton AB ...",Mill Woods Common,NaN,4b05e5a3f964a5205ce522e3
6,Tim Hortons,Coffee Shop,5204 23rd Ave,53.455062,-113.422306,"[{'label': 'display', 'lat': 53.45506170888434...",697,T6L 6Y7,CA,Edmonton,AB,Canada,"[5204 23rd Ave (Beside Mr. Lube), Edmonton AB ...",Beside Mr. Lube,NaN,4b3bcc6df964a520667b25e3
7,TD Canada Trust,Bank,2505 Hewes Way,53.455963,-113.424155,"[{'label': 'display', 'lat': 53.4559632, 'lng'...",547,T6L 6W6,CA,Edmonton,AB,Canada,"[2505 Hewes Way (by Mill Woods Town Centre), E...",by Mill Woods Town Centre,NaN,4b3d160ef964a5209c8d25e3
8,Subway (Mainstreet),Sandwich Place,NaN,53.459090,-113.433058,"[{'label': 'display', 'lat': 53.45908999332906...",351,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN,4c9bfeaed3c2b60c2113b4bc
9,Pizza Hut,Pizza Place,6504 28TH Ave,53.458756,-113.432715,"[{'label': 'display', 'lat': 53.458756, 'lng':...",346,T6L 6N3,CA,Edmonton,AB,Canada,"[6504 28TH Ave, Edmonton AB T6L 6N3, Canada]",NaN,NaN,4b91aa2bf964a52028ce33e3


In [114]:
dataframe_filtered.groupby('categories').count()

,name,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
categories,,,,,,,,,,,,,,,
BBQ Joint,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Bank,3,3,3,3,3,3,3,3,3,3,3,3,2,0,3
Business Service,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Coffee Shop,7,7,7,7,7,7,6,7,7,7,7,7,6,0,7
Community Center,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Convenience Store,2,2,2,2,2,2,2,2,2,2,2,2,1,0,2
Fast Food Restaurant,8,7,8,8,8,8,6,8,7,7,8,8,0,1,8
Gas Station,2,2,2,2,2,2,2,2,2,2,2,2,0,0,2
Golf Course,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1


In [115]:
print('There are {} uniques categories.'.format(len(dataframe_filtered['categories'].unique())))

There are 23 uniques categories.


In [53]:
from folium import plugins

venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

venues = plugins.MarkerCluster().add_to(venues_map)



# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.Marker(
        [lat, lng],
        icon=None,
        popup=label,
        ).add_to(venues_map)

# display map
venues_map

## Conclusion-
#### From the data on Indian population in Edmonton - Mill Woods, ratings of different Indian Restaurants and trending venues, it can be seen that opening of Indian Restaurant in the area of Mill Woods, Edmonton will be a Profitable business.